In [ ]:
# Installing Pyspark if not already present
# !pip install pyspark

In [3]:
# Importing SparkSesssion 
from pyspark.sql import SparkSession

In [4]:
# Create a SparkSession (Entry point of Spark)
spark = SparkSession.builder \
    .appName("DataCleaning") \
    .getOrCreate()


In [5]:
# Create a Pyspark DataFrame from 
df = spark.read.csv(r"D:\PROJECT\CSV_Files\Raw_DataSet.csv", header=True, inferSchema=True)


In [6]:
# Printing Schema of Data Frame and Displaying first 5 rows
df.printSchema()
df.show(5) 


root
 |-- State: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- GeneralHealth: string (nullable = true)
 |-- PhysicalHealthDays: double (nullable = true)
 |-- MentalHealthDays: double (nullable = true)
 |-- LastCheckupTime: string (nullable = true)
 |-- PhysicalActivities: string (nullable = true)
 |-- SleepHours: double (nullable = true)
 |-- RemovedTeeth: string (nullable = true)
 |-- HadHeartAttack: string (nullable = true)
 |-- HadAngina: string (nullable = true)
 |-- HadStroke: string (nullable = true)
 |-- HadAsthma: string (nullable = true)
 |-- HadSkinCancer: string (nullable = true)
 |-- HadCOPD: string (nullable = true)
 |-- HadDepressiveDisorder: string (nullable = true)
 |-- HadKidneyDisease: string (nullable = true)
 |-- HadArthritis: string (nullable = true)
 |-- HadDiabetes: string (nullable = true)
 |-- DeafOrHardOfHearing: string (nullable = true)
 |-- BlindOrVisionDifficulty: string (nullable = true)
 |-- DifficultyConcentrating: string (nullable = t

In [17]:
# Get all distinct values in the State column
distinct_states = df.select("State").distinct()

# Count the number of distinct values
distinct_states_count = distinct_states.count()

# Collect the distinct values as a list
all_distinct_states = distinct_states.collect()

# Print the count of distinct values
print(f"Count of distinct values in the State column: {distinct_states_count}")

# Print all distinct values
for state in all_distinct_states:
    print(state["State"])


Count of distinct values in the State column: 59
Hawaii
Arkansas
Connecticut
Calfornia
Illinois
District of Columbia
Delaware
Alaska
Albama
Georgia
Alabama
Arizona
Iowa
Florida
Indiana
Idaho
Huwaii
California
Colorado
Minnesota
Nebraska
Nevada
Missouri
Montana
Michigan
Kansas
Maryland
Massachusetts
Kentucky
Louisiana
Mississippi
New Hampshire
Maine
Utah
Ohio
Oregon
Texas
North Dakota
Pennsylvania
Oklahoma
New Mexico
Rhode Island
North Carolina
New Jersey
Tennessee
South Carolina
South Dakota
New York
Vermont
Puerto Rico
Washington
Virgin Islands
Vormont
West Virginia
Virginia
Guam
Wyoming
Wisconsin
None


In [19]:
# Filter out rows where State is None (null)
cleaned_df = df.filter(df["State"].isNotNull())

# Show the resulting DataFrame
# cleaned_df.show()

# Get all distinct values in the State column from the cleaned DataFrame
distinct_states_cleaned = cleaned_df.select("State").distinct()

# Collect the distinct values as a list
all_distinct_states_cleaned = distinct_states_cleaned.collect()

# Count the number of distinct values
distinct_states_count_cleaned = len(all_distinct_states_cleaned)

# Print the count of distinct values
print(f"Count of distinct values in the State column after removing None: {distinct_states_count_cleaned}")

# Print all distinct values
print("Distinct states:")
for state in all_distinct_states_cleaned:
    print(state["State"])



Count of distinct values in the State column after removing None: 58
Distinct states:
Hawaii
Arkansas
Connecticut
Calfornia
Illinois
District of Columbia
Delaware
Alaska
Albama
Georgia
Alabama
Arizona
Iowa
Florida
Indiana
Idaho
Huwaii
California
Colorado
Minnesota
Nebraska
Nevada
Missouri
Montana
Michigan
Kansas
Maryland
Massachusetts
Kentucky
Louisiana
Mississippi
New Hampshire
Maine
Utah
Ohio
Oregon
Texas
North Dakota
Pennsylvania
Oklahoma
New Mexico
Rhode Island
North Carolina
New Jersey
Tennessee
South Carolina
South Dakota
New York
Vermont
Puerto Rico
Washington
Virgin Islands
Vormont
West Virginia
Virginia
Guam
Wyoming
Wisconsin


In [20]:
#Other way out of doing above transforamtions and action

# Get distinct values in the State column from the cleaned DataFrame
distinct_states_cleaned = cleaned_df.select("State").distinct()

# Count the number of distinct values
distinct_states_count_cleaned = distinct_states_cleaned.count()

# Print the count of distinct values
print(f"Count of distinct values in the State column after removing None: {distinct_states_count_cleaned}")

# Optionally, print all distinct values
all_distinct_states_cleaned = distinct_states_cleaned.collect()
for state in all_distinct_states_cleaned:
    print(state["State"])


Count of distinct values in the State column after removing None: 58
Hawaii
Arkansas
Connecticut
Calfornia
Illinois
District of Columbia
Delaware
Alaska
Albama
Georgia
Alabama
Arizona
Iowa
Florida
Indiana
Idaho
Huwaii
California
Colorado
Minnesota
Nebraska
Nevada
Missouri
Montana
Michigan
Kansas
Maryland
Massachusetts
Kentucky
Louisiana
Mississippi
New Hampshire
Maine
Utah
Ohio
Oregon
Texas
North Dakota
Pennsylvania
Oklahoma
New Mexico
Rhode Island
North Carolina
New Jersey
Tennessee
South Carolina
South Dakota
New York
Vermont
Puerto Rico
Washington
Virgin Islands
Vormont
West Virginia
Virginia
Guam
Wyoming
Wisconsin


In [21]:
from pyspark.sql.functions import when

# Correcting the typos in the State column
df_cleaned = df.withColumn(
    "State",
    when(df["State"] == "Calfornia", "California")
    .when(df["State"] == "Albama", "Alabama")
    .when(df["State"] == "Huwaii", "Hawaii")
    .when(df["State"] == "Vormont", "Vermont")
    .otherwise(df["State"])
)

# Check the distinct values again
df_cleaned.select("State").distinct().show(truncate=False)


+--------------------+
|State               |
+--------------------+
|Hawaii              |
|Arkansas            |
|Connecticut         |
|Illinois            |
|District of Columbia|
|Delaware            |
|Alaska              |
|Georgia             |
|Alabama             |
|Arizona             |
|Iowa                |
|Florida             |
|Indiana             |
|Idaho               |
|California          |
|Colorado            |
|Minnesota           |
|Nebraska            |
|Nevada              |
|Missouri            |
+--------------------+
only showing top 20 rows



In [25]:
from pyspark.sql.functions import col, when, round

# Clean, cap, round, and standardize PhysicalHealthDays and MentalHealthDays columns
df_cleaned = df.withColumn(
    "PhysicalHealthDays",
    round(
        when(col("PhysicalHealthDays") < 0, 0)
        .when(col("PhysicalHealthDays") > 30, 30)
        .otherwise(col("PhysicalHealthDays"))
    ).cast("int")
)

df_cleaned = df_cleaned.withColumn(
    "MentalHealthDays",
    round(
        when(col("MentalHealthDays") < 0, 0)
        .when(col("MentalHealthDays") > 30, 30)
        .otherwise(col("MentalHealthDays"))
    ).cast("int")
)

# Show the cleaned and standardized columns
df_cleaned.select("PhysicalHealthDays", "MentalHealthDays").show()


+------------------+----------------+
|PhysicalHealthDays|MentalHealthDays|
+------------------+----------------+
|                 4|               0|
|                 0|               0|
|                 0|            NULL|
|                 6|               0|
|                 3|              12|
|                 0|               0|
|                 3|               0|
|                 5|               0|
|                 2|               0|
|                 0|               0|
|                 0|               0|
|                 3|               5|
|                 6|               0|
|                 0|               0|
|                29|              23|
|                 0|              14|
|                 0|               0|
|                 0|               0|
|                 0|               0|
|                29|               0|
+------------------+----------------+
only showing top 20 rows



In [27]:
#Sleep_Hours Standardizing
df_cleaned = df_cleaned.withColumn(
    "SleepHours",
    round(
        when(col("SleepHours") < 0, 0)
        .when(col("SleepHours") > 24,24)
        .otherwise(col("SleepHours"))
    ).cast("int")
)

# Show the cleaned and standardized columns
df_cleaned.select("SleepHours").show()

+----------+
|SleepHours|
+----------+
|         9|
|         5|
|         6|
|         7|
|         5|
|         7|
|         9|
|         7|
|         6|
|         6|
|         9|
|         4|
|         5|
|         6|
|         6|
|         7|
|         7|
|        11|
|         5|
|         9|
+----------+
only showing top 20 rows



In [28]:
distinct_values = df_cleaned.select("HadDiabetes").distinct()
distinct_values.show()


+--------------------+
|         HadDiabetes|
+--------------------+
|No, pre-diabetes ...|
|                  No|
|                 Yes|
|Yes, but only dur...|
+--------------------+



In [29]:
# Group by 'HadDiabetics' column and count occurrences
value_counts = df_cleaned.groupBy("HadDiabetes").count()

# Show the results
value_counts.show()


+--------------------+------+
|         HadDiabetes| count|
+--------------------+------+
|No, pre-diabetes ...|  5551|
|                  No|210337|
|                 Yes| 34718|
|Yes, but only dur...|  2023|
+--------------------+------+



In [30]:
# Filter to keep only "Yes" and "No"
filtered_df = df_cleaned.filter(df_cleaned["HadDiabetes"].isin(["Yes", "No"]))

# Count occurrences of each unique value in the filtered DataFrame
value_counts = filtered_df.groupBy("HadDiabetes").count()

# Show the results
value_counts.show()
value_counts.count()

+-----------+------+
|HadDiabetes| count|
+-----------+------+
|         No|210337|
|        Yes| 34718|
+-----------+------+



2

In [31]:
from pyspark.sql.functions import col, count

# Group by all columns and count occurrences
duplicates_df = df_cleaned.groupBy(df_cleaned.columns).count().filter(col("count") > 1)

# Calculate the total number of duplicate rows
duplicate_count = duplicates_df.select(count("count")).collect()[0][0]

print(f"Number of duplicate rows: {duplicate_count}")


Number of duplicate rows: 702


In [32]:
# Drop duplicate rows
deduplicated_df = df_cleaned.dropDuplicates()

# Optionally, you can check the number of rows in the deduplicated DataFrame
deduplicated_count = deduplicated_df.count()

# Show the result or save it as needed
deduplicated_df.show()
print(f"Number of rows after dropping duplicates: {deduplicated_count}")

+-------+------+-------------+------------------+----------------+--------------------+------------------+----------+--------------------+--------------+---------+---------+---------+-------------+-------+---------------------+----------------+------------+--------------------+-------------------+-----------------------+-----------------------+-----------------+-------------------------+-----------------+--------------------+--------------------+---------+---------------------+---------------+--------------+-----------------+-----+---------------+----------+------------+-------------+--------------------+----------------+--------+
|  State|   Sex|GeneralHealth|PhysicalHealthDays|MentalHealthDays|     LastCheckupTime|PhysicalActivities|SleepHours|        RemovedTeeth|HadHeartAttack|HadAngina|HadStroke|HadAsthma|HadSkinCancer|HadCOPD|HadDepressiveDisorder|HadKidneyDisease|HadArthritis|         HadDiabetes|DeafOrHardOfHearing|BlindOrVisionDifficulty|DifficultyConcentrating|DifficultyWalki

In [57]:
#Col with null values
from pyspark.sql.functions import col, count, when

# Count the number of null values for each column
null_counts = deduplicated_df.select([
    count(when(col(c).isNull(), c)).alias(c) for c in deduplicated_df.columns
])

# Collect the results into a dictionary
null_counts_dict = null_counts.collect()[0].asDict()

# Filter columns with null values
columns_with_nulls = [col for col, count in null_counts_dict.items() if count > 0]

print("Columns with null values:", columns_with_nulls)


Columns with null values: ['State', 'GeneralHealth', 'MentalHealthDays', 'LastCheckupTime', 'PhysicalActivities', 'RemovedTeeth']


In [34]:
#Forward-Fill NULL values

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, last, row_number
from pyspark.sql.window import Window

# Initialize Spark session
spark = SparkSession.builder.appName("ForwardFillingNullValues").getOrCreate()

# Add an Ordering Column based on Weight
ordered_df = deduplicated_df.withColumn(
    "order", 
    row_number().over(Window.orderBy("WeightInKilograms"))
)

# Define the Window Specification for forward-filling
window_spec = Window.orderBy("order").rowsBetween(Window.unboundedPreceding, Window.currentRow)

# List of columns to forward-fill
columns_to_fill = ['State', 'GeneralHealth', 'MentalHealthDays', 'LastCheckupTime', 'PhysicalActivities', 'RemovedTeeth']

# Apply Forward-Fill
forward_filled_df = ordered_df
for column in columns_to_fill:
    forward_filled_df = forward_filled_df.withColumn(
        column,
        last(column, ignorenulls=True).over(window_spec)
    )

# Drop the ordering column if not needed
forward_filled_df = forward_filled_df.drop("order")

# Show the resulting DataFrame
forward_filled_df.show()


+--------------+------+-------------+------------------+----------------+--------------------+------------------+----------+--------------------+--------------+---------+---------+---------+-------------+-------+---------------------+----------------+------------+-----------+-------------------+-----------------------+-----------------------+-----------------+-------------------------+-----------------+--------------------+--------------------+---------+---------------------+---------------+--------------+-----------------+-----+---------------+----------+------------+-------------+--------------------+----------------+--------+
|         State|   Sex|GeneralHealth|PhysicalHealthDays|MentalHealthDays|     LastCheckupTime|PhysicalActivities|SleepHours|        RemovedTeeth|HadHeartAttack|HadAngina|HadStroke|HadAsthma|HadSkinCancer|HadCOPD|HadDepressiveDisorder|HadKidneyDisease|HadArthritis|HadDiabetes|DeafOrHardOfHearing|BlindOrVisionDifficulty|DifficultyConcentrating|DifficultyWalking|D

In [35]:
from pyspark.sql.functions import col, count

# Check for null values in each column
null_counts = forward_filled_df.select([count(when(col(c).isNull(), c)).alias(c) for c in forward_filled_df.columns])

# Show the count of null values for each column
null_counts.show()


+-----+---+-------------+------------------+----------------+---------------+------------------+----------+------------+--------------+---------+---------+---------+-------------+-------+---------------------+----------------+------------+-----------+-------------------+-----------------------+-----------------------+-----------------+-------------------------+-----------------+------------+---------------+---------+---------------------+-----------+--------------+-----------------+---+---------------+----------+------------+-------------+-----------------+----------------+--------+
|State|Sex|GeneralHealth|PhysicalHealthDays|MentalHealthDays|LastCheckupTime|PhysicalActivities|SleepHours|RemovedTeeth|HadHeartAttack|HadAngina|HadStroke|HadAsthma|HadSkinCancer|HadCOPD|HadDepressiveDisorder|HadKidneyDisease|HadArthritis|HadDiabetes|DeafOrHardOfHearing|BlindOrVisionDifficulty|DifficultyConcentrating|DifficultyWalking|DifficultyDressingBathing|DifficultyErrands|SmokerStatus|ECigaretteUsage|

In [36]:
# Display the first 5 rows of the DataFrame
forward_filled_df.show(5)


+--------------+------+-------------+------------------+----------------+--------------------+------------------+----------+--------------------+--------------+---------+---------+---------+-------------+-------+---------------------+----------------+------------+-----------+-------------------+-----------------------+-----------------------+-----------------+-------------------------+-----------------+--------------------+--------------------+---------+---------------------+------------+--------------+-----------------+-----+---------------+----------+------------+-------------+--------------------+----------------+--------+
|         State|   Sex|GeneralHealth|PhysicalHealthDays|MentalHealthDays|     LastCheckupTime|PhysicalActivities|SleepHours|        RemovedTeeth|HadHeartAttack|HadAngina|HadStroke|HadAsthma|HadSkinCancer|HadCOPD|HadDepressiveDisorder|HadKidneyDisease|HadArthritis|HadDiabetes|DeafOrHardOfHearing|BlindOrVisionDifficulty|DifficultyConcentrating|DifficultyWalking|Diff

In [37]:
pd_save = forward_filled_df.toPandas()

In [38]:
import os
import pandas as pd

# Directory path
directory = 'D:/PROJECT/CSV_Files'

# Ensure the directory exists
if not os.path.exists(directory):
    os.makedirs(directory)

# Save to CSV
file_path = os.path.join(directory, 'preprocessed_dataset(complete).csv')
pd_save.to_csv(file_path, index=False)

print(f"File saved successfully to {file_path}")

# # Save to Excel (requires openpyxl or xlsxwriter)
# pandas_df.to_excel(output_path, index=False)

# # Save to Parquet
# pandas_df.to_parquet(output_path, index=False)


File saved successfully to D:/PROJECT/CSV_Files\preprocessed_dataset(complete).csv
